In [3]:
from google.colab import files

uploaded = files.upload()

Saving A01T_slice.mat to A01T_slice.mat
Saving A02T_slice.mat to A02T_slice.mat
Saving A03T_slice.mat to A03T_slice.mat
Saving A04T_slice.mat to A04T_slice.mat
Saving A05T_slice.mat to A05T_slice.mat
Saving A06T_slice.mat to A06T_slice.mat
Saving A07T_slice.mat to A07T_slice.mat
Saving A08T_slice.mat to A08T_slice.mat
Saving A09T_slice.mat to A09T_slice.mat


In [0]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell

In [0]:
def one_hot(i):
    a = [0,0,0,0]
    a[i-769] = 1;
    return a

In [0]:
# Load subject data and store the reshaped data in X_sub[i], y_sub[i] 
X_sub = []
y_sub = []
for i in np.arange(9):
    data = h5py.File('A0'+str(i+1)+'T_slice.mat', 'r')
    X_sub.append(np.copy(data['image']))
    y_sub.append(np.copy(data['type']))
    X_sub[i] = X_sub[i][:, :22, :]
    X_sub[i] = X_sub[i].transpose([0,2,1])
    y_sub[i] = y_sub[i][0,0:X_sub[i].shape[0]:1]
    y_sub[i] = np.asarray(y_sub[i], dtype=np.int32)
    y_sub[i] = [one_hot(j) for j in y_sub[i]]
    y_sub[i] = np.asarray(y_sub[i], dtype = np.int8)

In [0]:
# Remove NaN
for i in np.arange(9):
    y_sub[i] = y_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]
    X_sub[i] = X_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]

In [0]:
# Data Normalization
def normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/ sigma

In [11]:
# For training across all subjects
# Shuffle the data, sample 20% for testing, and the rest for training
X = []
y = []

X = np.concatenate([X_sub[i] for i in np.arange(9)])
y = np.concatenate([y_sub[i] for i in np.arange(9)])
num_trial = X.shape[0]
index_shuffle = np.arange(num_trial)
np.random.shuffle(index_shuffle)
X_shuffle = X[index_shuffle]
y_shuffle = y[index_shuffle]

training_index = int(np.floor(num_trial * (1 - 0.2)))
#validation_index = training_index + int(np.floor(num_trial * 0.2))
# test_index = num_trial

X_train = X_shuffle[0:training_index]
y_train = y_shuffle[0:training_index]
# X_val = X_shuffle[training_index:validation_index]
# y_val = y_shuffle[training_index:validation_index]
# y_val_steps = np.tile(y_val,((X_train.shape[1]),1))
X_test = X_shuffle[training_index:num_trial]
y_test = y_shuffle[training_index:num_trial]
y_test_steps = np.tile(y_test,((X_train.shape[1]),1))

X_train = normalize(X_train)
#X_val = normalize(X_val)
X_test = normalize(X_test)

print(X_train.shape)
print(X_test.shape)
print(y_test_steps.shape)

(2046, 1000, 22)
(512, 1000, 22)
(512000, 4)


In [12]:
# For training within each subject[i]
# Shuffle the data, sample 50 trials for testing, and the rest for training -->
X_train_sub = []
y_train_sub = []
X_test_sub = []
y_test_sub = []
y_test_steps_sub = []

for i in np.arange(9):
    num_trial_sub = X_sub[i].shape[0]
    index_shuffle_sub = np.arange(num_trial_sub)
    np.random.shuffle(index_shuffle_sub)
    X_shuffle_sub = X_sub[i][index_shuffle_sub]
    y_shuffle_sub = y_sub[i][index_shuffle_sub]
    
    training_index_sub = num_trial_sub - 50
    
    X_train_sub.append(X_shuffle_sub[0:training_index_sub])
    y_train_sub.append(y_shuffle_sub[0:training_index_sub])
    X_test_sub.append(X_shuffle[training_index_sub:num_trial_sub])
    y_test_sub.append(y_shuffle[training_index_sub:num_trial_sub])
    y_test_steps_sub.append(np.tile(y_test_sub[i],((X_train_sub[i].shape[1]),1)))

    X_train_sub[i] = normalize(X_train_sub[i])
    X_test_sub[i] = normalize(X_test_sub[i])

    print(X_train_sub[i].shape)
    print(X_test_sub[i].shape)
    print(y_test_steps_sub[i].shape)    

(237, 1000, 22)
(50, 1000, 22)
(50000, 4)
(236, 1000, 22)
(50, 1000, 22)
(50000, 4)
(236, 1000, 22)
(50, 1000, 22)
(50000, 4)
(234, 1000, 22)
(50, 1000, 22)
(50000, 4)
(232, 1000, 22)
(50, 1000, 22)
(50000, 4)
(235, 1000, 22)
(50, 1000, 22)
(50000, 4)
(238, 1000, 22)
(50, 1000, 22)
(50000, 4)
(232, 1000, 22)
(50, 1000, 22)
(50000, 4)
(228, 1000, 22)
(50, 1000, 22)
(50000, 4)


In [0]:
# For training across all subjects: Hyperparameters setting
tf.reset_default_graph()

learning_rate = 0.0001
training_epochs = 30
batch_size = 35
total_batches = (X_train.shape[0]//batch_size)
# print(total_batches)

alpha = 0.3
reg = 1

# Common parameters
n_hidden = 64
n_layers = 5

In [0]:
# For training within each subject: Hyperparameters setting
tf.reset_default_graph()

learning_rate_sub = 0.001
training_epochs_sub = 10
batch_size_sub = 15
total_batches_sub = (X_train_sub[i].shape[0]//batch_size_sub)
# print(total_batches)

alpha_sub = 0.3
reg_sub = 1

# Common parameters
n_hidden = 128
n_layers = 1

In [0]:
# Fixed Parameters
n_input = 22
n_steps = 1000
n_classes = 4


# Placeholders
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])
y_steps = tf.placeholder("float", [None, n_classes])
dropout = tf.placeholder(tf.float32) # prob to drop the cell

In [0]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def LSTM(x, weight, bias):
    cells = []
    for i in np.arange(n_layers):
        cell = rnn_cell.LSTMCell(n_hidden,state_is_tuple = True)
        cell = tf.contrib.rnn.DropoutWrapper(
            cell, output_keep_prob=1.0 - dropout)
        cells.append(cell)
    multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    output, state = tf.nn.dynamic_rnn(multi_layer_cell, x, dtype = tf.float32)
    output_flattened = tf.reshape(output, [-1, n_hidden])
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    output_all = tf.nn.sigmoid(output_logits)
    output_reshaped = tf.reshape(output_all,[-1,n_steps,n_classes])
    output_last = tf.gather(tf.transpose(output_reshaped,[1,0,2]), n_steps - 1)  

    return output_last, output_all

In [0]:
weight = weight_variable([n_hidden,n_classes])
bias = bias_variable([n_classes])
y_last, y_all = LSTM(x,weight,bias)

In [43]:
all_steps_cost = -tf.reduce_mean((y_steps * tf.log(y_all))  + (1 - y_steps) * tf.log(1 - y_all))
last_step_cost = -tf.reduce_mean((y * tf.log(y_last)) + ((1 - y) * tf.log(1 - y_last)))
regularizer = tf.nn.l2_loss(weight)
loss_function = (alpha * all_steps_cost) + ((1 - alpha) * last_step_cost) + reg * 0.5 *  regularizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(y_last, axis = 1),tf.argmax(y,axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [44]:
# Training across all subjects

display_step = 15 

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (y_train.shape[0] - batch_size)
            batch_x = X_train[offset:(offset + batch_size), :]
            batch_y = y_train[offset:(offset + batch_size), :]
            batch_y_steps = np.tile(batch_y,((X_train.shape[1]),1))
            
            _, acc, loss = session.run([optimizer, accuracy, loss_function], feed_dict={
                x: batch_x, y: batch_y, y_steps: batch_y_steps, dropout: 0.3})
            
            if (b % display_step == 0):
                print('training accuracy = %.2f, training loss = %f' % (acc, loss))
            
        test_accuracy = session.run(accuracy, feed_dict={
            x:X_test, y:y_test, y_steps: y_test_steps, dropout: 0})
        print('Epoch %d: test accuracy = %f' % (epoch, test_accuracy))
            
    

training accuracy = 0.17, training loss = 1.177835
training accuracy = 0.29, training loss = 1.157749
training accuracy = 0.20, training loss = 1.133192
training accuracy = 0.29, training loss = 1.077551
Epoch 0: test accuracy = 0.244141
training accuracy = 0.00, training loss = 1.041223
training accuracy = 0.29, training loss = 0.994378
training accuracy = 0.26, training loss = 0.980977
training accuracy = 0.26, training loss = 0.966106
Epoch 1: test accuracy = 0.244141
training accuracy = 0.20, training loss = 0.973138
training accuracy = 0.14, training loss = 0.951133
training accuracy = 0.26, training loss = 0.941801
training accuracy = 0.20, training loss = 0.927961
Epoch 2: test accuracy = 0.244141
training accuracy = 0.29, training loss = 0.910603
training accuracy = 0.31, training loss = 0.908740
training accuracy = 0.23, training loss = 0.897314
training accuracy = 0.26, training loss = 0.889641
Epoch 3: test accuracy = 0.244141
training accuracy = 0.17, training loss = 0.8930

Epoch 12: test accuracy = 0.255859
training accuracy = 0.23, training loss = 0.686812
training accuracy = 0.40, training loss = 0.678259
training accuracy = 0.17, training loss = 0.683847
training accuracy = 0.31, training loss = 0.671566
Epoch 13: test accuracy = 0.255859
training accuracy = 0.29, training loss = 0.672825
training accuracy = 0.11, training loss = 0.675848
training accuracy = 0.20, training loss = 0.671636
training accuracy = 0.29, training loss = 0.658352
Epoch 14: test accuracy = 0.238281
training accuracy = 0.17, training loss = 0.674203
training accuracy = 0.17, training loss = 0.660951
training accuracy = 0.17, training loss = 0.661977
training accuracy = 0.43, training loss = 0.649511
Epoch 15: test accuracy = 0.238281
training accuracy = 0.43, training loss = 0.649765
training accuracy = 0.31, training loss = 0.649991
training accuracy = 0.23, training loss = 0.653788
training accuracy = 0.37, training loss = 0.638804
Epoch 16: test accuracy = 0.238281
training 

training accuracy = 0.23, training loss = 0.601588
Epoch 25: test accuracy = 0.238281
training accuracy = 0.23, training loss = 0.601891
training accuracy = 0.34, training loss = 0.596790
training accuracy = 0.17, training loss = 0.599117
training accuracy = 0.51, training loss = 0.592210
Epoch 26: test accuracy = 0.238281
training accuracy = 0.17, training loss = 0.598652
training accuracy = 0.23, training loss = 0.595620
training accuracy = 0.31, training loss = 0.594115
training accuracy = 0.29, training loss = 0.593212
Epoch 27: test accuracy = 0.238281
training accuracy = 0.23, training loss = 0.593503
training accuracy = 0.29, training loss = 0.592939
training accuracy = 0.37, training loss = 0.590249
training accuracy = 0.26, training loss = 0.594391
Epoch 28: test accuracy = 0.238281
training accuracy = 0.17, training loss = 0.591991
training accuracy = 0.20, training loss = 0.592266
training accuracy = 0.17, training loss = 0.591866
training accuracy = 0.20, training loss = 0.

In [28]:
# Training within each subject

sub_num = [1, 2, 3, 4, 5, 6, 7, 8, 9]
#sub_num = [1]

display_step_sub = 3

for i in sub_num:

    with tf.Session() as session:
        tf.global_variables_initializer().run()
        for epoch in range(training_epochs_sub):
            for b in range(total_batches_sub):    
                offset = (b * batch_size_sub) % (y_train_sub[i-1].shape[0] - batch_size_sub)
                batch_x = X_train_sub[i-1][offset:(offset + batch_size_sub), :]
                batch_y = y_train_sub[i-1][offset:(offset + batch_size_sub), :]
                batch_y_steps = np.tile(batch_y,((X_train_sub[i-1].shape[1]),1))
                
                _, acc, loss = session.run([optimizer, accuracy, loss_function], feed_dict={
                    x: batch_x, y: batch_y, y_steps: batch_y_steps, dropout: 0.3})
                
                if (b % display_step_sub == 0):
                    print('For subject %d : training accuracy = %.2f, training loss = %f' % (i, acc, loss))

            test_accuracy = session.run(accuracy, feed_dict={
                x:X_test_sub[i-1], y:y_test_sub[i-1], y_steps: y_test_steps_sub[i-1], dropout: 0})
            print('For subject %d, Epoch %d: test accuracy = %f' % (i, epoch, test_accuracy))

For subject 1 : training accuracy = 0.33, training loss = 1.773301
For subject 1 : training accuracy = 0.13, training loss = 1.726514
For subject 1 : training accuracy = 0.20, training loss = 1.698125
For subject 1 : training accuracy = 0.33, training loss = 1.660046
For subject 1 : training accuracy = 0.27, training loss = 1.618992
For subject 1, Epoch 0: test accuracy = 0.220000
For subject 1 : training accuracy = 0.33, training loss = 1.587766
For subject 1 : training accuracy = 0.27, training loss = 1.507805
For subject 1 : training accuracy = 0.27, training loss = 1.507824
For subject 1 : training accuracy = 0.40, training loss = 1.467749
For subject 1 : training accuracy = 0.27, training loss = 1.451481
For subject 1, Epoch 1: test accuracy = 0.220000
For subject 1 : training accuracy = 0.13, training loss = 1.418364
For subject 1 : training accuracy = 0.27, training loss = 1.361368
For subject 1 : training accuracy = 0.33, training loss = 1.351876
For subject 1 : training accura

For subject 1 : training accuracy = 0.20, training loss = 0.881302
For subject 1 : training accuracy = 0.53, training loss = 0.853499
For subject 1 : training accuracy = 0.53, training loss = 0.849175
For subject 1 : training accuracy = 0.27, training loss = 0.854439
For subject 1, Epoch 9: test accuracy = 0.240000
For subject 2 : training accuracy = 0.13, training loss = 1.709210
For subject 2 : training accuracy = 0.33, training loss = 1.657349
For subject 2 : training accuracy = 0.27, training loss = 1.612431
For subject 2 : training accuracy = 0.00, training loss = 1.606781
For subject 2 : training accuracy = 0.47, training loss = 1.541178
For subject 2, Epoch 0: test accuracy = 0.260000
For subject 2 : training accuracy = 0.27, training loss = 1.498422
For subject 2 : training accuracy = 0.33, training loss = 1.456198
For subject 2 : training accuracy = 0.27, training loss = 1.411043
For subject 2 : training accuracy = 0.20, training loss = 1.390569
For subject 2 : training accura

For subject 2 : training accuracy = 0.33, training loss = 0.860406
For subject 2 : training accuracy = 0.40, training loss = 0.836912
For subject 2 : training accuracy = 0.40, training loss = 0.831908
For subject 2, Epoch 8: test accuracy = 0.300000
For subject 2 : training accuracy = 0.40, training loss = 0.831129
For subject 2 : training accuracy = 0.33, training loss = 0.809819
For subject 2 : training accuracy = 0.33, training loss = 0.823688
For subject 2 : training accuracy = 0.53, training loss = 0.793543
For subject 2 : training accuracy = 0.53, training loss = 0.800150
For subject 2, Epoch 9: test accuracy = 0.280000
For subject 3 : training accuracy = 0.27, training loss = 1.707825
For subject 3 : training accuracy = 0.20, training loss = 1.669967
For subject 3 : training accuracy = 0.20, training loss = 1.630469
For subject 3 : training accuracy = 0.07, training loss = 1.615786
For subject 3 : training accuracy = 0.13, training loss = 1.550590
For subject 3, Epoch 0: test ac

For subject 3 : training accuracy = 0.33, training loss = 0.905249
For subject 3 : training accuracy = 0.40, training loss = 0.898200
For subject 3, Epoch 7: test accuracy = 0.420000
For subject 3 : training accuracy = 0.27, training loss = 0.881635
For subject 3 : training accuracy = 0.40, training loss = 0.862780
For subject 3 : training accuracy = 0.40, training loss = 0.870773
For subject 3 : training accuracy = 0.47, training loss = 0.846056
For subject 3 : training accuracy = 0.27, training loss = 0.861581
For subject 3, Epoch 8: test accuracy = 0.400000
For subject 3 : training accuracy = 0.33, training loss = 0.843329
For subject 3 : training accuracy = 0.47, training loss = 0.833338
For subject 3 : training accuracy = 0.20, training loss = 0.822993
For subject 3 : training accuracy = 0.40, training loss = 0.816996
For subject 3 : training accuracy = 0.40, training loss = 0.818115
For subject 3, Epoch 9: test accuracy = 0.360000
For subject 4 : training accuracy = 0.33, trainin

For subject 4 : training accuracy = 0.13, training loss = 0.982140
For subject 4, Epoch 6: test accuracy = 0.320000
For subject 4 : training accuracy = 0.40, training loss = 0.967833
For subject 4 : training accuracy = 0.27, training loss = 0.963097
For subject 4 : training accuracy = 0.40, training loss = 0.942683
For subject 4 : training accuracy = 0.20, training loss = 0.943654
For subject 4 : training accuracy = 0.20, training loss = 0.930178
For subject 4, Epoch 7: test accuracy = 0.300000
For subject 4 : training accuracy = 0.33, training loss = 0.922444
For subject 4 : training accuracy = 0.20, training loss = 0.907017
For subject 4 : training accuracy = 0.47, training loss = 0.891554
For subject 4 : training accuracy = 0.07, training loss = 0.906838
For subject 4 : training accuracy = 0.40, training loss = 0.879285
For subject 4, Epoch 8: test accuracy = 0.300000
For subject 4 : training accuracy = 0.33, training loss = 0.887144
For subject 4 : training accuracy = 0.40, trainin

For subject 5, Epoch 5: test accuracy = 0.360000
For subject 5 : training accuracy = 0.40, training loss = 1.030113
For subject 5 : training accuracy = 0.27, training loss = 1.033421
For subject 5 : training accuracy = 0.20, training loss = 1.040413
For subject 5 : training accuracy = 0.33, training loss = 0.991532
For subject 5 : training accuracy = 0.33, training loss = 0.988204
For subject 5, Epoch 6: test accuracy = 0.320000
For subject 5 : training accuracy = 0.33, training loss = 0.969005
For subject 5 : training accuracy = 0.20, training loss = 0.979184
For subject 5 : training accuracy = 0.20, training loss = 0.959076
For subject 5 : training accuracy = 0.53, training loss = 0.942517
For subject 5 : training accuracy = 0.33, training loss = 0.946372
For subject 5, Epoch 7: test accuracy = 0.300000
For subject 5 : training accuracy = 0.20, training loss = 0.926266
For subject 5 : training accuracy = 0.33, training loss = 0.917445
For subject 5 : training accuracy = 0.27, trainin

For subject 6 : training accuracy = 0.47, training loss = 1.051718
For subject 6 : training accuracy = 0.33, training loss = 1.039658
For subject 6 : training accuracy = 0.60, training loss = 1.002065
For subject 6 : training accuracy = 0.47, training loss = 0.999678
For subject 6 : training accuracy = 0.40, training loss = 0.999760
For subject 6, Epoch 5: test accuracy = 0.260000
For subject 6 : training accuracy = 0.27, training loss = 0.990964
For subject 6 : training accuracy = 0.40, training loss = 0.978792
For subject 6 : training accuracy = 0.33, training loss = 0.964881
For subject 6 : training accuracy = 0.40, training loss = 0.941346
For subject 6 : training accuracy = 0.33, training loss = 0.945028
For subject 6, Epoch 6: test accuracy = 0.260000
For subject 6 : training accuracy = 0.47, training loss = 0.916066
For subject 6 : training accuracy = 0.27, training loss = 0.935764
For subject 6 : training accuracy = 0.40, training loss = 0.909321
For subject 6 : training accura

For subject 7 : training accuracy = 0.13, training loss = 1.140265
For subject 7 : training accuracy = 0.40, training loss = 1.095043
For subject 7 : training accuracy = 0.33, training loss = 1.092986
For subject 7 : training accuracy = 0.40, training loss = 1.067327
For subject 7, Epoch 4: test accuracy = 0.280000
For subject 7 : training accuracy = 0.47, training loss = 1.050393
For subject 7 : training accuracy = 0.27, training loss = 1.065185
For subject 7 : training accuracy = 0.33, training loss = 1.029014
For subject 7 : training accuracy = 0.07, training loss = 1.038504
For subject 7 : training accuracy = 0.47, training loss = 0.994631
For subject 7, Epoch 5: test accuracy = 0.320000
For subject 7 : training accuracy = 0.47, training loss = 0.989919
For subject 7 : training accuracy = 0.00, training loss = 1.010390
For subject 7 : training accuracy = 0.53, training loss = 0.956251
For subject 7 : training accuracy = 0.47, training loss = 0.955468
For subject 7 : training accura

For subject 8 : training accuracy = 0.33, training loss = 1.166561
For subject 8 : training accuracy = 0.13, training loss = 1.166468
For subject 8 : training accuracy = 0.27, training loss = 1.130982
For subject 8, Epoch 3: test accuracy = 0.260000
For subject 8 : training accuracy = 0.53, training loss = 1.102225
For subject 8 : training accuracy = 0.40, training loss = 1.082417
For subject 8 : training accuracy = 0.47, training loss = 1.068055
For subject 8 : training accuracy = 0.07, training loss = 1.077761
For subject 8 : training accuracy = 0.20, training loss = 1.058654
For subject 8, Epoch 4: test accuracy = 0.340000
For subject 8 : training accuracy = 0.47, training loss = 1.026806
For subject 8 : training accuracy = 0.40, training loss = 1.016895
For subject 8 : training accuracy = 0.47, training loss = 0.996564
For subject 8 : training accuracy = 0.13, training loss = 1.018199
For subject 8 : training accuracy = 0.20, training loss = 0.995803
For subject 8, Epoch 5: test ac

For subject 9 : training accuracy = 0.27, training loss = 1.292430
For subject 9 : training accuracy = 0.33, training loss = 1.255715
For subject 9, Epoch 2: test accuracy = 0.240000
For subject 9 : training accuracy = 0.47, training loss = 1.222588
For subject 9 : training accuracy = 0.27, training loss = 1.207095
For subject 9 : training accuracy = 0.27, training loss = 1.205673
For subject 9 : training accuracy = 0.27, training loss = 1.189424
For subject 9 : training accuracy = 0.33, training loss = 1.163534
For subject 9, Epoch 3: test accuracy = 0.220000
For subject 9 : training accuracy = 0.47, training loss = 1.141999
For subject 9 : training accuracy = 0.40, training loss = 1.127613
For subject 9 : training accuracy = 0.27, training loss = 1.108346
For subject 9 : training accuracy = 0.47, training loss = 1.105422
For subject 9 : training accuracy = 0.27, training loss = 1.091610
For subject 9, Epoch 4: test accuracy = 0.280000
For subject 9 : training accuracy = 0.27, trainin